In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [3]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test':  transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.48216, 0.44653],[0.24703, 0.24349, 0.26159])
    ])
}

trainset = torchvision.datasets.CIFAR10(root='./',
                    train=True,download=True,
                    transform=data_transforms['train'])
valset=torchvision.datasets.CIFAR10(root='./',
                  train=True,download=True,
                  transform=data_transforms['val'])

indices = np.arange(50000)
np.random.shuffle(indices)
train_loader = torch.utils.data.DataLoader(trainset,batch_size=500, shuffle=False, sampler=torch.utils.data.SubsetRandomSampler(indices[:45000]))
val_loader = torch.utils.data.DataLoader(valset,batch_size=500, shuffle=False, sampler=torch.utils.data.SubsetRandomSampler(indices[45000:50000]))



Files already downloaded and verified
Files already downloaded and verified


In [0]:
class_names = trainset.classes
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataloaders={'train':train_loader,'val':val_loader}
dataset_sizes = {'train':45000,'val':5000}

In [5]:
class_names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [0]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            
            print(phase,':',time.strftime("%Y--%m--%d %H:%M:%S", time.localtime(int(time.time()))))
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
#                 print('2:',time.time())
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
#                     print('3:',time.time())
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
#                     print('4:',time.time())
                    loss = criterion(outputs, labels)
#                     print('5:',time.time())
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [0]:
resnet18 = models.resnet18(pretrained=True)

#for param in resnet18.parameters():
#    param.requires_grad = False
    
# Parameters of newly sconstructed modules have requires_grad=True by default
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs,10)
optimizer = optim.SGD(resnet18.parameters(), lr=0.001, momentum=0.9)  # 随机梯度下降
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(alex.parameters(), lr=3e-4)
# # Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [8]:
train_model(resnet18.to(device), criterion, optimizer, exp_lr_scheduler, num_epochs=10)

Epoch 0/9
----------
train : 2020--06--11 08:32:16


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


train Loss: 1.7561 Acc: 0.4050
val : 2020--06--11 08:36:27
val Loss: 1.0130 Acc: 0.7018

Epoch 1/9
----------
train : 2020--06--11 08:36:45
train Loss: 1.1048 Acc: 0.6321
val : 2020--06--11 08:40:51
val Loss: 0.6477 Acc: 0.7954

Epoch 2/9
----------
train : 2020--06--11 08:41:09
train Loss: 0.8885 Acc: 0.6974
val : 2020--06--11 08:45:14
val Loss: 0.5060 Acc: 0.8338

Epoch 3/9
----------
train : 2020--06--11 08:45:31
train Loss: 0.7820 Acc: 0.7314
val : 2020--06--11 08:49:23
val Loss: 0.4121 Acc: 0.8652

Epoch 4/9
----------
train : 2020--06--11 08:49:38
train Loss: 0.7170 Acc: 0.7540
val : 2020--06--11 08:53:25
val Loss: 0.3571 Acc: 0.8854

Epoch 5/9
----------
train : 2020--06--11 08:53:40
train Loss: 0.6750 Acc: 0.7667
val : 2020--06--11 08:57:27
val Loss: 0.3189 Acc: 0.8938

Epoch 6/9
----------
train : 2020--06--11 08:57:42
train Loss: 0.6445 Acc: 0.7789
val : 2020--06--11 09:01:29
val Loss: 0.3141 Acc: 0.8986

Epoch 7/9
----------
train : 2020--06--11 09:01:44
train Loss: 0.6443 A

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  